# SSG닷컴 이마트몰 카테고리별 베스트 상품 크롤링

# 사용 패키지

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import time
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

# 사전작업
- 1. 가격정보의 중복되는 데이터를 제거 및 불필요한 특수문자 제거를 위한 함수
- 2. 가격정보에서 쉼표 제거하는 함수
 - 1번 함수의 예시 
  - '할인적용가\n3,840\n원\n(￦3,840)'
  - 필요한 가격정보인 6~14부분만 슬라이싱한다. 
  - 할인정보의 따로 작업을 하지 않아도 되지만, 할인 가격정보의 인덱스가 필요할 경우를 대비해 함수에서는 따로 구분을 해준다.

- 3. 한 가지 제품 정보를 두 줄로 표현한 경우, 브랜드를 한줄로 표기하고 해당 인덱스를 제거해준다.

In [2]:
# 할인가격 중복 제거
# em.ssg_price 는 할인가격이 리스트에 별도로 기록되기 때문에, 정제를 하기 매우 어렵다
# div.cunit_price.notranslate : 여기에는 기존가격, 할인가격이 한줄로 표시되어 가공이 더 수월하다
def remove_number(x):
    for i in range(len(x)):
        # if문이 없어도 되지만, 할인정보가 있는 행을 구분하기 위해 넣어줬다.
        if '에누리' in x[i]:
            x[i] = x[i][0:25]
        x[i] = x[i][6:14].replace('\n','').replace('원','').replace('(','').replace('\\','').replace('￦','').replace('~','')
    return x

# 가격에 쉼표 제거
def remove_str(x):
    return x.replace(',','')

# product 가져올때 브랜드이름, 상품명 따로 표기된 리스트가 있다.
# [브랜드 이름 + 제품명] 의 형태로 변경하고, 브랜드 이름 인덱스를 제거한다.
# 추가되는 상품이 있을시, name_list에 추가만하면 된다.
def replace_name(x):
    name_list = ['오뚜기','농심','팔도','PEACOCK','풀무원','No Brand','삼양','맥심','동서식품(주)','서울우유','매일','코카콜라',
                 '크라운','오리온','오리온제과','해태','롯데','E-MART FRESH','빙그레','하겐다즈','나뚜루','하림','맛있닭','테미즈',
                 '교촌1991','오븐에빠진닭','에그파파','허닭','올품','매일유업','서울우유','남양유업',
                 'LG생활건강','다우니','헨켈','참그린','크리넥스','코디','이마트24','SHYRILLA','유한킴벌리','모나리자','깨끗한나라',
                 '수려한','더랩바이블랑두', '스킨푸드', '니베아','피지오겔','SCINIC','더바디샵','한스킨','온더바디','스웨덴 에그팩',
                 '바이오더마','셀리맥스','버츠비','안나홀츠','설화수','홀리카홀리카','아이오페','바디판타지','센카','Dr.G','콜마운틴',
                 '에뛰드하우스','이지바이','트렉스타','도이터','카멜백','사츠','파라고나(paragona)','아크테릭스','레키',
                 '디스커버리(discovery)','트렉스타레저타임','파쉬','SUPERFEET','[JAJU/자주]신세계인터내셔날','라스포르티바[LASPORTIVA]',
                 '캠프라인','넥스토치','제이린드버그','레드페이스','오스프리','스탠리','월드컵(WORLDCUP)','데카트론','마운틴스미스',
                 '우리가스토리','콜핑','이드나인(ID9)','K2(케이투)','제이큐','반스플라이','BFL','마운티아[MOUNTIA]','휴몬트',
                 '헬리콘텍스','핫한날','선데이에프터눈즈','피크나인','언더아머(UNDER ARMOUR)','듀이셀','블리스텍스','네파[NEPA]',
                 '블랙야크','닉왁스(NIKWAX)','아이비','씨투써밋','부리나','마운틴가이드','BUCK703','팩세이프(pacsafe)',
                 '로우알파인','엘캡','바주카','헬리콘텍스[Helikon-Tex]','버팔로','트레스패스','마우나케아','3M','웨스트우드','아로마티카'
                 '머렐','노스페이스(NORTHFACE)','마몽드','성분에디터','카밀','수월한'
                ]
            
    for i in range(len(x)-1):
        for name in name_list:
            if x[i] == name:
                x[i+1] = (name + ' ' + x[i+1])

    for i in range(len(x)):
        for name in name_list:
            try:
                if x[i] == name:
                    x.remove(name)
            except : continue

# 크롤링
- 총 12개 품목에 대한 best-list 데이터 프레임 생성 후 하나로 통합
- ✰ 한번에 2개 이상 카테고리 작업시, 데이터 로드가 되지 않아서 각 카테고리 작업별로 '20초' 간격을 둔다.
- 페이지가 많은 '뷰티케어', '등산용품'의 경우 동일 카테고리간 데이터 로드가 되지 않을 때가 있어서, 5페이지마다 '5초'씩 간격을 둔다

In [4]:
# 인기품목별 BEST list & link
best_list=['라면','과자','커피','제철과일','아이스크림','돼지고기','닭고기','우유','세제','화장지','뷰티케어','등산용품']
best_list_link=['1000020108','1000020109','1000020110','1000020111','1000020112','1000020113','1000020114','1000020115','1000020116','1000020117','1000020118','1000020119']

# 현재 카테고리별 best상품을 모두 담아줄 데이터프레임 생성
best_total_df=pd.DataFrame()

# 품목별로 순환
for link in best_list_link:

    # 해당 카테고리 넣어둘 df 생성
    total_df=pd.DataFrame()

    # 한 카테고리 내에서 실행하는 반복문, 페이지수 20으로 설정
    for num in range(20):
        if (num == 5) or (num == 10) or (num == 15): # 5페이지마다 한번씩 더 쉬어준다.
            time.sleep(5)
        else:    
            # url지정
            url=f'http://emart.ssg.com/best/ajaxGetMoreItemList.ssg?cornrId=1000016472&cornrSetId={link}&dispCtgId=&pageSize=12&page={num+1}'
            # 여기서 link는 라면, 과자, 커피 등의 정보이고 num은 동일 품목에서 더보기를 누를 때 나오는 페이지 정보다

            # requests 정보 설정
            # 현재 PC의 referer과 User-Agent 기입
            headers = { 
                        'referer': 'http://emart.ssg.com/best/main.ssg?Egnb=best',
                        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36'
                      }

            seed = np.random.randint(100)
            np.random.seed(seed)    
            a = np.random.randint(5)
            time.sleep(a)
            try:
                response=requests.get(url, headers=headers)
            except:
                print('접속 실패')
                break

            # html 전환
            data = BeautifulSoup(response.text, 'html.parser')

            # 페이지별 이름, 가격, 평점 리스트 생성
            product = [name.text for name in data.select('em.tx_ko')]
            replace_name(product)   # 윗줄에서 정의한 제품명을 함수를 사용해 중복 데이터를 날려준다
            
            price = [coin.text.strip() for coin in data.select('div.cunit_price.notranslate')]
            remove_number(price)   # 위에서 정의한 함수를 사용해서 가격정보를 다듬는다.(할인으로 인한 중복 가격표기도 제거)
            
            rate = [star.text.strip().strip().replace('별점 ','').replace('점','') for star in data.select('div.rate_bg')]
            # 별점을 시각화나 데이터 작업에 용이하게 숫자만 남겨둔다
            
            # 페이지 정보가 없으면 현재 검색중인 카테고리 루프문 종료
            if not product:
                break;

            else:
                # 현재 페이지 데이터프레임 생성(1개의 카테고리)
                df = pd.DataFrame(columns=['product','price','rate'], data=list(zip(product,price,rate)))

                # price, 별점 정보를 숫자형으로 전환
                df['price'] = df['price'].apply(remove_str).astype(int)
                df['rate'] = df['rate'].astype(float)

                # 현재 페이지 데이터프레임를 현재 카테고리 데이터프레임에 업데이트
                temp = df
                total_df = pd.concat([total_df, temp])
                total_df['카테고리'] = f'{best_list[best_list_link.index(link)]}'
                
    # 카테고리별 작업 완료
    print(f'{best_list[best_list_link.index(link)]} 인기품목 크롤링 완료')

    # 전체 데이터프레임에 완료된 카테고리 데이터 업데이트
    total_temp = total_df
    best_total_df = pd.concat([best_total_df, total_temp])
    best_total_df.index = range(len(best_total_df.index)) # 인덱스 초기화
    best_total_df = best_total_df[['카테고리','product','price','rate']]
    print(f'전체 데이터에 {best_list[best_list_link.index(link)]} 카테고리 병합 완료')
    
    if link != '1000020119':
        print('다음 카테고리 크롤링 진행\n')
        time.sleep(20)         # 페이지 막힘 방지를 위해 20초 뒤에 다음 카테고리 실행
    else:
        print('***** 크롤링 작업 완료 *****\n')

# 전체 카테고리 작업분 저장
best_total_df.to_csv('/Users/ppangppang/Desktop/ssac/02.Python_/Python_Basic_Class/practice/210929_Best_product.csv',index=False)
print('***** 데이터 저장 완료 *****')

라면 인기품목 크롤링 완료
전체 데이터에 라면 카테고리 병합 완료
다음 카테고리 크롤링 진행

과자 인기품목 크롤링 완료
전체 데이터에 과자 카테고리 병합 완료
다음 카테고리 크롤링 진행

커피 인기품목 크롤링 완료
전체 데이터에 커피 카테고리 병합 완료
다음 카테고리 크롤링 진행

제철과일 인기품목 크롤링 완료
전체 데이터에 제철과일 카테고리 병합 완료
다음 카테고리 크롤링 진행

아이스크림 인기품목 크롤링 완료
전체 데이터에 아이스크림 카테고리 병합 완료
다음 카테고리 크롤링 진행

돼지고기 인기품목 크롤링 완료
전체 데이터에 돼지고기 카테고리 병합 완료
다음 카테고리 크롤링 진행

닭고기 인기품목 크롤링 완료
전체 데이터에 닭고기 카테고리 병합 완료
다음 카테고리 크롤링 진행

우유 인기품목 크롤링 완료
전체 데이터에 우유 카테고리 병합 완료
다음 카테고리 크롤링 진행

세제 인기품목 크롤링 완료
전체 데이터에 세제 카테고리 병합 완료
다음 카테고리 크롤링 진행

화장지 인기품목 크롤링 완료
전체 데이터에 화장지 카테고리 병합 완료
다음 카테고리 크롤링 진행

뷰티케어 인기품목 크롤링 완료
전체 데이터에 뷰티케어 카테고리 병합 완료
다음 카테고리 크롤링 진행

등산용품 인기품목 크롤링 완료
전체 데이터에 등산용품 카테고리 병합 완료
***** 크롤링 작업 완료 *****

***** 데이터 저장 완료 *****


In [5]:
# 데이터를 확인해보자
best_total_df

,카테고리,product,price,rate
0,라면,[농심] 올리브 짜파게티 (140gx5입),3670,4.95
1,라면,[농심] 신라면 120gx5입,3670,4.95
2,라면,오뚜기 진라면 매운맛 120g*5입,3100,4.96
3,라면,[농심] 육개장 사발면 (86gx6입),4490,4.95
4,라면,[농심] 안성탕면 5입 625g (125g5입),3280,4.94
...,...,...,...,...
747,등산용품,스탠리 GO 진공 슬림 보틀 280ml 보온보냉 텀블러,27000,4.00
748,등산용품,트라이탄 루프통 1.0리터 수통,17100,4.29
749,등산용품,아이비 냉풍조끼 W-A2006 (흰색),82824,5.00
750,등산용품,등산용품/캠핑용품/아이젠/힙색/등산스틱,2400,5.00


In [6]:
# 오류발생시 현재까지 작업된것 수기로 확인해보자
# best_total_df.to_csv('/Users/ppangppang/Desktop/ssac/02.Python_/Python_Basic_Class/practice/210929_Best_product.csv', index=False)
# best_total_df

In [7]:
# 저장한 데이터를 불러와 보자
df = pd.read_csv('/Users/ppangppang/Desktop/ssac/02.Python_/Python_Basic_Class/practice/210929_Best_product.csv')
df

,카테고리,product,price,rate
0,라면,[농심] 올리브 짜파게티 (140gx5입),3670,4.95
1,라면,[농심] 신라면 120gx5입,3670,4.95
2,라면,오뚜기 진라면 매운맛 120g*5입,3100,4.96
3,라면,[농심] 육개장 사발면 (86gx6입),4490,4.95
4,라면,[농심] 안성탕면 5입 625g (125g5입),3280,4.94
...,...,...,...,...
747,등산용품,스탠리 GO 진공 슬림 보틀 280ml 보온보냉 텀블러,27000,4.00
748,등산용품,트라이탄 루프통 1.0리터 수통,17100,4.29
749,등산용품,아이비 냉풍조끼 W-A2006 (흰색),82824,5.00
750,등산용품,등산용품/캠핑용품/아이젠/힙색/등산스틱,2400,5.00


## 유의사항
- 위에서 언급한 것처럼, 특정 브랜드가 새로 생길경우, 데이터가 밀려서 입력될 경우, 사전작업 탭의 3번 항목을 업데이트해주자
- 서버 접속때문인지, 한개의 카테고리가 그 양이 많을 경우, 서버가 막혀서 저장이 안될 경우가 있다.
- 위의 코드는 카테고리별로 25초의 간격을 두었으나, 데이터가 많은 등산용품의 마지막 페이지가 잘리는 경우가 생긴다. -> 데이터 중간중간 time.sleep을 걸어주는 것도 방법이겠다

#(부록) 코드 정보 및 코드 테스트
- 아래는 함수 지정 및 정상 작동 여부를 위해 따로 남겨둔 공간이다.
- 함수나 리스트 지정이 복잡해서 몇가지 예시를 남겨둔다.

In [693]:
url=' http://emart.ssg.com/best/ajaxGetMoreItemList.ssg?cornrId=1000016472&cornrSetId=1000020109&dispCtgId=&pageSize=12&page=3'


# requests 정보 설정
headers = {
            'referer': 'http://emart.ssg.com/best/main.ssg?Egnb=best',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36'
}

response=requests.get(url, headers=headers)

# html 전환
data = BeautifulSoup(response.text, 'html.parser')

# 페이지별 이름, 가격, 평점 리스트 생성
product = [name.text for name in data.select('em.tx_ko')]
price = [coin.text for coin in data.select('em.ssg_price')] 

In [694]:
# em.ssg_price 이걸로 가격을 가져올경우, 할인가격과 기본가격이 따로 나눠져서 작업이 어렵다
price3 = [coin.text.strip() for coin in data.select('div.cunit_price.notranslate')]
price3

['할인적용가\n1,200\n원\n(￦1,200)',
 '할인적용가\n3,840\n원\n(￦3,840)',
 '할인적용가\n2,988\n원\n(￦2,988)\n\n\n판매가\n4,980\n원\n\n\n가격 상세보기\n\n\n판매가\n4,980\n\n\n에누리1,992원\n\n\n최적가\n2,988',
 '할인적용가\n980\n원\n(￦980)',
 '할인적용가\n980\n원\n(￦980)',
 '할인적용가\n4,380\n원\n(￦4,380)',
 '할인적용가\n3,550\n원\n(￦3,550)',
 '할인적용가\n3,830\n원\n(￦3,830)',
 '할인적용가\n1,200\n원\n(￦1,200)',
 '할인적용가\n6,980\n원\n(￦6,980)',
 '할인적용가\n1,200\n원\n(￦1,200)',
 '할인적용가\n4,780\n원\n(￦4,780)']

In [ ]:
ff=data.select('div.cunit_price.notranslate')
a=20
price3 = [coin.text.strip() for coin in data.select('div.cunit_price.notranslate')]
for i in range(len(price3)):
    if '에누리' in price3[i]:
        print(i)
        price3[i] = price3[i][0:25]
    price3[i] = price3[i][6:14].replace('\n','').replace('원','').replace('(','').replace('\\','').replace('￦','')
    
price3

In [ ]:
def remove_number(x):
    for i in range(len(x)):
        if '에누리' in x[i]:
            x[i] = x[i][0:25]
        x[i] = x[i][6:14].replace('\n','').replace('원','').replace('(','')
    return x

In [ ]:
ab=[1,2,3,4,5]
test=[1,2]
del ab[test[1]]

In [ ]:
time.sleep(10)
print(ab)

In [ ]:
# 링크 및 인덱스 호출 테스트
a=['a','b','c','d']
b=[1,2,3,4]

for i in a:
    print(f'tt{b[a.index(i)]}')
    
a.index('b')

In [ ]:
# product 정리 기존 코드

# product 가져올때 브랜드이름, 상품명 따로 표기된 리스트에서 브랜드 이름 붙여주기
# [브랜드 이름 + 그 브랜드 제품명] 의 형태는 고정으로 되어 있으므로, 제품명에 브랜드 이름을 붙인 뒤, 
# 위치가 어쨌든 리스트 슬라이싱보다 직접 그 내용을 지워주는게 더 효과적이다
def replace_name(x):
    # 불러오는 한 페이지의 리스트 길이만큼 반복한다
    for i in range(len(x)):
        
        # 라면
        if x[i] == '오뚜기':
            x[i+1] = ('오뚜기 ' + x[i+1])
        elif x[i] == 'PEACOCK':
            x[i+1] = ('PEACOCK ' + x[i+1])
        elif x[i] == '농심':
            x[i+1] = ('농심 ' + x[i+1])
        elif x[i] == '팔도':
            x[i+1] = ('팔도 ' + x[i+1])
        elif x[i] == '풀무원':
            x[i+1] = ('풀무원 ' + x[i+1])
        elif x[i] == 'No Brand':
            x[i+1] = ('No Brand ' + x[i+1])
        elif x[i] == '삼양':
            x[i+1] = ('삼양 ' + x[i+1])
    
        # 과자
        elif x[i] == '크라운':
            x[i+1] = ('크라운 ' + x[i+1])
        elif x[i] == '오리온':
            x[i+1] = ('오리온 ' + x[i+1])
        elif x[i] == '오리온제과':
            x[i+1] = ('오리온제과 ' + x[i+1])    
        elif x[i] == '해태':
            x[i+1] = ('해태 ' + x[i+1])
        elif x[i] == '롯데':
            x[i+1] = ('롯데 ' + x[i+1])
            
        # 커피
        elif x[i] == '맥심':
            x[i+1] = ('맥심 ' + x[i+1])
        elif x[i] == '동서식품(주)':
            x[i+1] = ('동서식품(주) ' + x[i+1])
        elif x[i] == '서울우유':
            x[i+1] = ('서울우유 ' + x[i+1])    
        elif x[i] == '매일':
            x[i+1] = ('매일 ' + x[i+1])
        elif x[i] == '코카콜라':
            x[i+1] = ('코카콜라 ' + x[i+1])
            
        # 과일
        elif x[i] == 'E-MART FRESH':
            x[i+1] = ('E-MART FRESH ' + x[i+1])

        # 아이스크림
        elif x[i] == '빙그레':
            x[i+1] = ('빙그레 ' + x[i+1])
        elif x[i] == '하겐다즈':
            x[i+1] = ('하겐다즈 ' + x[i+1])
        elif x[i] == '나뚜루':
            x[i+1] = ('나뚜루 ' + x[i+1])    

        # (돼지고기)
        
        # 닭고기
        elif x[i] == '하림':
            x[i+1] = ('하림 ' + x[i+1])
        elif x[i] == '맛있닭':
            x[i+1] = ('맛있닭 ' + x[i+1])
        elif x[i] == '테미즈':
            x[i+1] = ('테미즈 ' + x[i+1])   
        elif x[i] == '교촌1991':
            x[i+1] = ('교촌1991 ' + x[i+1])   
        elif x[i] == '오븐에빠진닭':
            x[i+1] = ('오븐에빠진닭 ' + x[i+1])   
        elif x[i] == '에그파파':
            x[i+1] = ('에그파파 ' + x[i+1])   
        elif x[i] == '허닭':
            x[i+1] = ('허닭 ' + x[i+1])
        elif x[i] == '올품':
            x[i+1] = ('올품 ' + x[i+1])   

        # 우유
        elif x[i] == '매일유업':
            x[i+1] = ('매일유업 ' + x[i+1])
        elif x[i] == '서울우유':
            x[i+1] = ('서울우유 ' + x[i+1])
        elif x[i] == '남양유업':
            x[i+1] = ('남양유업 ' + x[i+1])

        # 세제
        elif x[i] == 'LG생활건강':
            x[i+1] = ('LG생활건강 ' + x[i+1])
        elif x[i] == '다우니':
            x[i+1] = ('다우니 ' + x[i+1])
        elif x[i] == '헨켈':
            x[i+1] = ('헨켈 ' + x[i+1])   
        elif x[i] == '참그린':
            x[i+1] = ('참그린 ' + x[i+1])   

        # 화장지
        elif x[i] == '크리넥스':
            x[i+1] = ('크리넥스 ' + x[i+1])
        elif x[i] == '코디':
            x[i+1] = ('코디 ' + x[i+1])
        elif x[i] == '이마트24':
            x[i+1] = ('이마트24 ' + x[i+1])   
        elif x[i] == 'SHYRILLA':
            x[i+1] = ('SHYRILLA ' + x[i+1])  
        elif x[i] == '유한킴벌리':
            x[i+1] = ('유한킴벌리 ' + x[i+1])  
        elif x[i] == '모나리자':
            x[i+1] = ('모나리자 ' + x[i+1])  
        elif x[i] == '깨끗한나라':
            x[i+1] = ('깨끗한나라 ' + x[i+1])  
        elif x[i] == '수려한':
            x[i+1] = ('수려한 ' + x[i+1])  

        # 뷰티케어
        elif x[i] == '더랩바이블랑두':
            x[i+1] = ('더랩바이블랑두 ' + x[i+1])
        elif x[i] == '스킨푸드':
            x[i+1] = ('스킨푸드 ' + x[i+1])
        elif x[i] == '니베아':
            x[i+1] = ('니베아 ' + x[i+1])
        elif x[i] == '피지오겔':
            x[i+1] = ('피지오겔 ' + x[i+1])
        elif x[i] == 'SCINIC':
            x[i+1] = ('SCINIC ' + x[i+1])   
        elif x[i] == '더바디샵':
            x[i+1] = ('더바디샵 ' + x[i+1])  
        elif x[i] == '한스킨':
            x[i+1] = ('한스킨 ' + x[i+1])  
        elif x[i] == '온더바디':
            x[i+1] = ('온더바디 ' + x[i+1]) 
        elif x[i] == '스웨덴 에그팩':
            x[i+1] = ('스웨덴 에그팩 ' + x[i+1])
        elif x[i] == '바이오더마':
            x[i+1] = ('바이오더마 ' + x[i+1]) 
        elif x[i] == '셀리맥스':
            x[i+1] = ('셀리맥스 ' + x[i+1]) 
        elif x[i] == '버츠비':
            x[i+1] = ('버츠비 ' + x[i+1]) 
        elif x[i] == '안나홀츠':
            x[i+1] = ('안나홀츠 ' + x[i+1])
        elif x[i] == '설화수':
            x[i+1] = ('설화수 ' + x[i+1]) 
        elif x[i] == '홀리카홀리카':
            x[i+1] = ('홀리카홀리카 ' + x[i+1])
        elif x[i] == '아이오페':
            x[i+1] = ('아이오페 ' + x[i+1]) 
        elif x[i] == '바디판타지':
            x[i+1] = ('바디판타지 ' + x[i+1])
        elif x[i] == '센카':
            x[i+1] = ('센카 ' + x[i+1])
        elif x[i] == 'Dr.G':
            x[i+1] = ('Dr.G ' + x[i+1])
        elif x[i] == '콜마운틴':
            x[i+1] = ('콜마운틴 ' + x[i+1])
        elif x[i] == '에뛰드하우스':
            x[i+1] = ('에뛰드하우스 ' + x[i+1])
            
        # 등산용품
        elif x[i] == '이지바이':
            x[i+1] = ('이지바이 ' + x[i+1])
        elif x[i] == '트렉스타':
            x[i+1] = ('트렉스타 ' + x[i+1])
        elif x[i] == '도이터':
            x[i+1] = ('도이터 ' + x[i+1])
        elif x[i] == '카멜백':
            x[i+1] = ('카멜백 ' + x[i+1])   
        elif x[i] == '사츠':
            x[i+1] = ('사츠 ' + x[i+1])  
        elif x[i] == '파라고나(paragona)':
            x[i+1] = ('파라고나(paragona) ' + x[i+1])  
        elif x[i] == '아크테릭스':
            x[i+1] = ('아크테릭스 ' + x[i+1]) 
        elif x[i] == '레키':
            x[i+1] = ('레키 ' + x[i+1])
        elif x[i] == '디스커버리(discovery)':
            x[i+1] = ('디스커버리(discovery) ' + x[i+1]) 
        elif x[i] == '트렉스타레저타임':
            x[i+1] = ('트렉스타레저타임 ' + x[i+1]) 
        elif x[i] == '파쉬':
            x[i+1] = ('파쉬 ' + x[i+1]) 
        elif x[i] == 'SUPERFEET':
            x[i+1] = ('SUPERFEET ' + x[i+1])
        elif x[i] == '[JAJU/자주]신세계인터내셔날':
            x[i+1] = ('[JAJU/자주]신세계인터내셔날 ' + x[i+1]) 
        elif x[i] == '라스포르티바[LASPORTIVA]':
            x[i+1] = ('라스포르티바[LASPORTIVA] ' + x[i+1])
        elif x[i] == '캠프라인':
            x[i+1] = ('캠프라인 ' + x[i+1]) 
        elif x[i] == '넥스토치':
            x[i+1] = ('넥스토치 ' + x[i+1])
        elif x[i] == '제이린드버그':
            x[i+1] = ('제이린드버그 ' + x[i+1]) 
        elif x[i] == '레드페이스':
            x[i+1] = ('레드페이스 ' + x[i+1]) 
        elif x[i] == '오스프리':
            x[i+1] = ('오스프리 ' + x[i+1])
        elif x[i] == '스탠리':
            x[i+1] = ('스탠리 ' + x[i+1]) 
        elif x[i] == '월드컵(WORLDCUP)':
            x[i+1] = ('월드컵(WORLDCUP) ' + x[i+1]) 
        elif x[i] == '데카트론':
            x[i+1] = ('데카트론 ' + x[i+1])
        elif x[i] == '마운틴스미스':
            x[i+1] = ('마운틴스미스 ' + x[i+1]) 
        elif x[i] == '우리가스토리':
            x[i+1] = ('우리가스토리 ' + x[i+1]) 
        elif x[i] == '콜핑':
            x[i+1] = ('콜핑 ' + x[i+1])
        elif x[i] == '이드나인(ID9)':
            x[i+1] = ('이드나인(ID9) ' + x[i+1])
        elif x[i] == 'K2(케이투)':
            x[i+1] = ('K2(케이투) ' + x[i+1])
        elif x[i] == '제이큐':
            x[i+1] = ('제이큐 ' + x[i+1])
        elif x[i] == '반스플라이':
            x[i+1] = ('반스플라이 ' + x[i+1]) 
        elif x[i] == 'BFL':
            x[i+1] = ('BFL ' + x[i+1]) 
        elif x[i] == '마운티아[MOUNTIA]':
            x[i+1] = ('마운티아[MOUNTIA] ' + x[i+1]) 
        elif x[i] == '휴몬트':
            x[i+1] = ('휴몬트 ' + x[i+1]) 
        elif x[i] == '헬리콘텍스':
            x[i+1] = ('헬리콘텍스 ' + x[i+1]) 
        elif x[i] == '핫한날':
            x[i+1] = ('핫한날 ' + x[i+1]) 
        elif x[i] == '선데이에프터눈즈':
            x[i+1] = ('선데이에프터눈즈 ' + x[i+1]) 
        elif x[i] == '피크나인':
            x[i+1] = ('피크나인 ' + x[i+1])           
        elif x[i] == '언더아머(UNDER ARMOUR)':
            x[i+1] = ('언더아머(UNDER ARMOUR) ' + x[i+1]) 
        elif x[i] == '듀이셀':
            x[i+1] = ('듀이셀 ' + x[i+1]) 
        elif x[i] == '블리스텍스':
            x[i+1] = ('블리스텍스 ' + x[i+1])
        elif x[i] == '네파[NEPA]':
            x[i+1] = ('네파[NEPA] ' + x[i+1]) 
        elif x[i] == '블랙야크':
            x[i+1] = ('블랙야크 ' + x[i+1])
        elif x[i] == '닉왁스(NIKWAX)':
            x[i+1] = ('닉왁스(NIKWAX) ' + x[i+1]) 
        elif x[i] == '아이비':
            x[i+1] = ('아이비 ' + x[i+1])
        elif x[i] == '씨투써밋':
            x[i+1] = ('씨투써밋 ' + x[i+1]) 
        elif x[i] == '부리나':
            x[i+1] = ('부리나 ' + x[i+1]) 
        elif x[i] == '마운틴가이드':
            x[i+1] = ('마운틴가이드 ' + x[i+1]) 
        elif x[i] == 'BUCK703':
            x[i+1] = ('BUCK703 ' + x[i+1])
        elif x[i] == '팩세이프(pacsafe)':
            x[i+1] = ('팩세이프(pacsafe) ' + x[i+1]) 
        elif x[i] == '로우알파인':
            x[i+1] = ('로우알파인 ' + x[i+1])
        elif x[i] == '엘캡':
            x[i+1] = ('엘캡 ' + x[i+1])
        elif x[i] == '바주카':
            x[i+1] = ('바주카 ' + x[i+1])
        elif x[i] == '헬리콘텍스[Helikon-Tex]':
            x[i+1] = ('헬리콘텍스[Helikon-Tex] ' + x[i+1])
        elif x[i] == '버팔로':
            x[i+1] = ('버팔로 ' + x[i+1])
        elif x[i] == '트레스패스':
            x[i+1] = ('트레스패스 ' + x[i+1])
        elif x[i] == '마우나케아':
            x[i+1] = ('마우나케아 ' + x[i+1])
        elif x[i] == '3M':
            x[i+1] = ('3M ' + x[i+1])


    # 인덱스 제거
    # len을 사용해서 반복문을 사용 할 경우, 리스트 길이의 변동으로 오류가 생길 수 있다.
    # 페이지는 10개를 거의 넘지 않고, 한 페이지당 많아야 5번 중복이 있으므로 넉넉하게 15번 정도만 반복해도 충분
    for i in range(15):
        # 라면
        if '오뚜기' in x:
            x.remove('오뚜기')
        elif 'PEACOCK'in x:
            x.remove('PEACOCK')
        elif '농심' in x:
            x.remove('농심')
        elif '팔도' in x:
            x.remove('팔도')
        elif 'No Brand' in x:
            x.remove('No Brand')
        elif '풀무원' in x:
            x.remove('풀무원')
        elif '삼양' in x:
            x.remove('삼양')

        # 과자
        elif '크라운' in x:
            x.remove('크라운')
        elif '오리온'in x:
            x.remove('오리온')
        elif '오리온제과' in x:
            x.remove('오리온제과')
        elif '해태' in x:
            x.remove('해태')
        elif '롯데' in x:
            x.remove('롯데')

        # 커피
        elif '맥심' in x:
            x.remove('맥심')
        elif '동서식품(주)'in x:
            x.remove('동서식품(주)')
        elif '서울우유' in x:
            x.remove('서울우유')
        elif '매일' in x:
            x.remove('매일')
        elif '코카콜라' in x:
            x.remove('코카콜라')

        # 과일
        elif 'E-MART FRESH' in x:
            x.remove('E-MART FRESH')

        # 아이스크림
        elif '빙그레' in x:
            x.remove('빙그레')
        elif '하겐다즈'in x:
            x.remove('하겐다즈')
        elif '나뚜루' in x:
            x.remove('나뚜루')

        # 닭고기
        elif '하림' in x:
            x.remove('하림')
        elif '맛있닭'in x:
            x.remove('맛있닭')
        elif '테미즈' in x:
            x.remove('테미즈')
        elif '교촌1991' in x:
            x.remove('교촌1991')
        elif '오븐에빠진닭' in x:
            x.remove('오븐에빠진닭')  
        elif '에그파파' in x:
            x.remove('에그파파')  
        elif '허닭' in x:
            x.remove('허닭')  
        elif '올품' in x:
            x.remove('올품')  

        # 우유
        elif '매일유업' in x:
            x.remove('매일유업')
        elif '서울우유'in x:
            x.remove('서울우유')
        elif '남양유업' in x:
            x.remove('남양유업')

        # 세제
        elif 'LG생활건강' in x:
            x.remove('LG생활건강')
        elif '다우니'in x:
            x.remove('다우니')
        elif '헨켈' in x:
            x.remove('헨켈')
        elif '참그린' in x:
            x.remove('참그린')

        # 화장지
        elif '크리넥스' in x:
            x.remove('크리넥스')
        elif '코디'in x:
            x.remove('코디')
        elif '이마트24' in x:
            x.remove('이마트24')
        elif 'SHYRILLA' in x:
            x.remove('SHYRILLA')
        elif '유한킴벌리' in x:
            x.remove('유한킴벌리')
        elif '모나리자' in x:
            x.remove('모나리자')
        elif '깨끗한나라' in x:
            x.remove('깨끗한나라')
        elif '수려한' in x:
            x.remove('수려한')

        # 뷰티케어
        elif '더랩바이블랑두' in x:
            x.remove('더랩바이블랑두')
        elif '스킨푸드' in x:
            x.remove('스킨푸드')
        elif '니베아' in x:
            x.remove('니베아')
        elif '피지오겔'in x:
            x.remove('피지오겔')
        elif 'SCINIC' in x:
            x.remove('SCINIC')
        elif '더바디샵' in x:
            x.remove('더바디샵')
        elif '한스킨' in x:
            x.remove('한스킨')
        elif '온더바디' in x:
            x.remove('온더바디')        
        elif '스웨덴 에그팩' in x:
            x.remove('스웨덴 에그팩')
        elif '바이오더마'in x:
            x.remove('바이오더마')
        elif '셀리맥스' in x:
            x.remove('셀리맥스')
        elif '버츠비' in x:
            x.remove('버츠비')
        elif '안나홀츠' in x:
            x.remove('안나홀츠')
        elif '설화수' in x:
            x.remove('설화수')         
        elif '홀리카홀리카' in x:
            x.remove('홀리카홀리카')    
        elif '아이오페' in x:
            x.remove('아이오페')    
        elif '바디판타지' in x:
            x.remove('바디판타지')    
        elif '센카' in x:
            x.remove('센카')
        elif 'Dr.G' in x:
            x.remove('Dr.G')  
        elif '콜마운틴' in x:
            x.remove('콜마운틴')  
        elif '에뛰드하우스' in x:
            x.remove('에뛰드하우스')  

        # 등산용품
        elif '이지바이' in x:
            x.remove('이지바이')
        elif '트렉스타' in x:
            x.remove('트렉스타')
        elif '도이터'in x:
            x.remove('도이터')
        elif '카멜백' in x:
            x.remove('카멜백')
        elif '사츠' in x:
            x.remove('사츠')
        elif '파라고나(paragona)' in x:
            x.remove('파라고나(paragona)')
        elif '아크테릭스' in x:
            x.remove('아크테릭스')        
        elif '레키' in x:
            x.remove('레키')
        elif '디스커버리(discovery)'in x:
            x.remove('디스커버리(discovery)')
        elif '트렉스타레저타임' in x:
            x.remove('트렉스타레저타임')
        elif '파쉬' in x:
            x.remove('파쉬')
        elif 'SUPERFEET' in x:
            x.remove('SUPERFEET')
        elif '[JAJU/자주]신세계인터내셔날' in x:
            x.remove('[JAJU/자주]신세계인터내셔날')         
        elif '라스포르티바[LASPORTIVA]' in x:
            x.remove('라스포르티바[LASPORTIVA]')    
        elif '캠프라인' in x:
            x.remove('캠프라인')    
        elif '넥스토치' in x:
            x.remove('넥스토치')    
        elif '제이린드버그' in x:
            x.remove('제이린드버그')  
        elif '레드페이스' in x:
            x.remove('레드페이스')
        elif '오스프리'in x:
            x.remove('오스프리')
        elif '스탠리' in x:
            x.remove('스탠리')
        elif '월드컵(WORLDCUP)' in x:
            x.remove('월드컵(WORLDCUP)')
        elif '데카트론' in x:
            x.remove('데카트론')
        elif '마운틴스미스' in x:
            x.remove('마운틴스미스')        
        elif '우리가스토리' in x:
            x.remove('우리가스토리')
        elif '콜핑'in x:
            x.remove('콜핑')
        elif '이드나인(ID9)' in x:
            x.remove('이드나인(ID9)')
        elif 'K2(케이투)' in x:
            x.remove('K2(케이투)')
        elif '제이큐' in x:
            x.remove('제이큐')
        elif '반스플라이' in x:
            x.remove('반스플라이')         
        elif 'BFL' in x:
            x.remove('BFL')    
        elif '마운티아[MOUNTIA]' in x:
            x.remove('마운티아[MOUNTIA]')    
        elif '휴몬트' in x:
            x.remove('휴몬트')    
        elif '헬리콘텍스' in x:
            x.remove('헬리콘텍스')  
        elif '핫한날' in x:
            x.remove('핫한날')    
        elif '선데이에프터눈즈' in x:
            x.remove('선데이에프터눈즈')    
        elif '피크나인' in x:
            x.remove('피크나인')    
        elif '언더아머(UNDER ARMOUR)' in x:
            x.remove('언더아머(UNDER ARMOUR)')     
        elif '듀이셀' in x:
            x.remove('듀이셀')   
        elif '블리스텍스' in x:
            x.remove('블리스텍스')   
        elif '네파[NEPA]' in x:
            x.remove('네파[NEPA]')   
        elif '블랙야크' in x:
            x.remove('블랙야크')   
        elif '닉왁스(NIKWAX)' in x:
            x.remove('닉왁스(NIKWAX)')   
        elif '아이비' in x:
            x.remove('아이비')   
        elif '씨투써밋' in x:
            x.remove('씨투써밋')   
        elif '부리나' in x:
            x.remove('부리나')   
        elif '마운틴가이드' in x:
            x.remove('마운틴가이드')   
        elif 'BUCK703' in x:
            x.remove('BUCK703')
        elif '팩세이프(pacsafe)' in x:
            x.remove('팩세이프(pacsafe)')  
        elif '로우알파인' in x:
            x.remove('로우알파인')  
        elif '엘캡' in x:
            x.remove('엘캡')  
        elif '바주카' in x:
            x.remove('바주카')  
        elif '헬리콘텍스[Helikon-Tex]' in x:
            x.remove('헬리콘텍스[Helikon-Tex]')      
        elif '버팔로' in x:
            x.remove('버팔로')    
        elif '트레스패스' in x:
            x.remove('트레스패스')    
        elif '마우나케아' in x:
            x.remove('마우나케아')    
        elif '3M' in x:
            x.remove('3M')    
            
    return x